# 📰 Building a news classifier with weak supervision

In this tutorial, we will build a news classifier using rules and weak supervision: 

- 📰 For this example, we use the AG News dataset but you can follow this process to programmatically label any dataset.
- 🤿 The train split without labels is used to build a training set with rules, Argilla and Snorkel's Label model.
- 🔧 The test set is used for evaluating our weak labels, label model and downstream news classifier.
- 🤯 We achieve 0.82 macro avg. f1-score without using a single example from the original dataset and using a pretty lightweight model (scikit-learn's `MultinomialNB`).

The following diagram shows the overall process for using Weak supervision with Argilla:

![labelling-textclassification-snorkel-weaksupervision](../../_static/tutorials/labelling-textclassification-snorkel-weaksupervision/labelling-textclassification-snorkel-weaksupervision.png)

## Introduction

> *Weak supervision is a branch of machine learning where noisy, limited, or imprecise sources are used to provide supervision signal for labeling large amounts of training data in a supervised learning setting. This approach alleviates the burden of obtaining hand-labeled data sets, which can be costly or impractical. Instead, inexpensive weak labels are employed with the understanding that they are imperfect, but can nonetheless be used to create a strong predictive model.* [[Wikipedia]](https://en.wikipedia.org/wiki/Weak_supervision)

For a broader introduction to weak supervision, as well as further references, we recommend the excellent [overview by Alex Ratner et al.](https://www.snorkel.org/blog/weak-supervision).

This tutorial aims to be a practical introduction to weak supervision and will walk you through its entire process.
First we will generate weak labels with *Argilla*, combine these labels with *Snorkel*, and finally train a classifier with *Scikit Learn*.

## Setup

For this tutorial we also need some third party libraries that can be installed via pip:

In [ ]:
%pip install snorkel datasets sklearn -qqq

<div class="alert alert-info">

Note

If you want to skip the first three sections of this tutorial, and only prepare the training set and train a downstream model, you can load the records directly from the [Hugging Face Hub](https://huggingface.co/datasets):

```python
import argilla as rg
from datasets import load_dataset

# this replaces the `records = label_model.predict()` line of section 4
records = rg.read_datasets(
    load_dataset("argilla/news", split="train"),
    task="TextClassification",
)
```

</div>

## 1. Load test and unlabelled datasets into Argilla

First, let's download the `ag_news` data set and have a quick look at it.

In [ ]:
from datasets import load_dataset

# load our data
dataset = load_dataset("ag_news")

# get the index to label mapping
labels = dataset["test"].features["label"].names


In [ ]:
import pandas as pd

# quick look at our data
with pd.option_context("display.max_colwidth", None):
    display(dataset["test"].to_pandas().head())


Now we will log the test split of our data set to *Argilla*, which we will be using for testing our label and downstream models.

In [ ]:
import argilla as rg

# build our test records
records = [
    rg.TextClassificationRecord(
        text=record["text"],
        metadata={"split": "test"},
        annotation=labels[record["label"]],
    )
    for record in dataset["test"]
]

# log the records to Argilla
rg.log(records, name="news")


In a second step we log the train split without labels.
Remember, our goal is to programmatically build a training set using rules and weak supervision.

In [ ]:
# build our training records without labels
records = [
    rg.TextClassificationRecord(
        text=record["text"],
        metadata={"split": "unlabelled"},
    )
    for record in dataset["train"]
]

# log the records to Argilla
rg.log(records, name="news")


The result of the above is the following dataset in Argilla, with **127,600 records** (120,000 unlabelled and 7,600 for testing). 

You can use the web app to find good rules for programmatic labeling!

## 2. Interactive weak labeling: Finding and defining rules

After logging the dataset, you can find and save rules directly with the UI. Then, you can read the rules with Python to train a label or downstream model, as we'll see in the next step. 

## 3. Denoise weak labels with Snorkel's Label Model

The goal at this step is to **denoise** the weak labels we've just created using rules. There are several approaches to this problem using different statistical methods.

In this tutorial, we're going to use Snorkel but you can actually use any other Label model or weak supervision method, such as FlyingSquid for example (see the [Weak supervision guide](../../guides/techniques/weak_supervision.html) for more details).
For convenience, Argilla defines a simple wrapper over Snorkel's Label Model so it's easier to use with Argilla weak labels and datasets

Let's first read the rules defined in our dataset and create our weak labels:

In [ ]:
from argilla.labeling.text_classification import WeakLabels

weak_labels = WeakLabels(dataset="news")


In [ ]:
weak_labels.summary()


In [ ]:
from argilla.labeling.text_classification import Snorkel

# create the label model
label_model = Snorkel(weak_labels)

# fit the model
label_model.fit()


In [ ]:
print(label_model.score(output_str=True))


## 4. Prepare our training set

Now, we already have a "denoised" training set, which we can prepare for training a downstream model. 
The label model predict returns `TextClassificationRecord` objects with the `predictions` from the label model. 

We can either refine and review these records using the Argilla web app, use them as is, or filter them by score, for example.

In this case, we assume the predictions are precise enough and use them without any revision. 
Our training set has ~38,000 records, which corresponds to all records where the label model has not abstained.

In [ ]:
import pandas as pd

# get records with the predictions from the label model
records = label_model.predict()
# you can replace this line with
# records = rg.read_datasets(
#    load_dataset("argilla/news", split="train"),
#    task="TextClassification",
# )

# we could also use the `weak_labels.label2int` dict
label2int = {"Sports": 0, "Sci/Tech": 1, "World": 2, "Business": 3}

# extract training data
X_train = [rec.text for rec in records]
y_train = [label2int[rec.prediction[0][0]] for rec in records]


In [ ]:
# quick look at our training data with the weak labels from our label model
with pd.option_context("display.max_colwidth", None):
    display(pd.DataFrame({"text": X_train, "label": y_train}))


## 5. Train a downstream model with scikit-learn

Now, let's train our final model using `scikit-learn`:

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# define our final classifier
classifier = Pipeline([("vect", CountVectorizer()), ("clf", MultinomialNB())])

# fit the classifier
classifier.fit(
    X=X_train,
    y=y_train,
)


To test our trained model, we use the records with validated annotations, that is the original ag_news test set.

In [ ]:
# retrieve records with annotations
test_ds = weak_labels.records(has_annotation=True)
# you can replace this line with
# test_ds = rg.read_datasets(
#    load_dataset("argilla/news_test", split="train"),
#    task="TextClassification",
# )

# extract text and labels
X_test = [rec.text for rec in test_ds]
y_test = [label2int[rec.annotation] for rec in test_ds]


In [ ]:
# compute the test accuracy
accuracy = classifier.score(
    X=X_test,
    y=y_test,
)

print(f"Test accuracy: {accuracy}")


Not too bad! 🥳

We have achieved around **0.82 accuracy** without even using a single example from the original `ag_news` train set and with a small set of 16 rules. 
Also, we've improved over the 0.75 accuracy of our Label Model.

Finally, let's take a look at more detailed metrics:

In [ ]:
from sklearn import metrics

# get predictions for the test set
predicted = classifier.predict(X_test)

print(metrics.classification_report(y_test, predicted, target_names=label2int.keys()))


At this point, we could go back to the UI to define more rules for those labels with less performance. Looking at the above table, we might want to add some more rules for increasing the recall of the `Business` label.

## Summary

In this tutorial, we saw how you can leverage weak supervision to quickly build up a large training data set, and use it for the training of a first lightweight model.

*Argilla* is a very handy tool to start the weak supervision process by making it easy to find a good set of starting rules, and to reiterate on them dynamically.
Since *Argilla* also provides built-in support for the most common label models, you can get from rules to weak labels in a few straight forward steps.
For more suggestions on how to leverage weak labels, you can checkout our [weak supervision guide](../../guides/techniques/weak_supervision.html) where we describe an [interesting approach](../../guides/techniques/weak_supervision.html#Joint-Model-with-Weasel) to jointly train the label and a transformers downstream model.

## Next steps

⭐ Argilla [Github repo](https://github.com/argilla-io/argilla) to stay updated.

📚 [Argilla documentation](https://docs.argilla.io) for more guides and tutorials.

🙋‍♀️ Join the Argilla community on [Slack](https://bit.ly/3o0Pfyk)

## Appendix I: Create rules and weak labels from Python

For some use cases, you might want to use Python for defining labeling rules and generating weak labels. Argilla provides you with the ability to define and test rules and labeling functions directly using Python. This might be useful for combining it with rules defined in the UI, and for leveraging structured resources such as lexicons and gazeteers which are easier to use directly a programmatic environment.

In this section, we define the rules we've defined in the UI, this time directly using Python:

In [ ]:
from argilla.labeling.text_classification import Rule

# define queries and patterns for each category (using ES DSL)
queries = [
    (["money", "financ*", "dollar*"], "Business"),
    (["war", "gov*", "minister*", "conflict"], "World"),
    (["footbal*", "sport*", "game", "play*"], "Sports"),
    (["sci*", "techno*", "computer*", "software", "web"], "Sci/Tech"),
]

# define rules
rules = [Rule(query=term, label=label) for terms, label in queries for term in terms]


In [ ]:
from argilla.labeling.text_classification import WeakLabels, add_rules

# generate the weak labels

add_rules(dataset="news")
weak_labels = WeakLabels(dataset="news")

If you want to apply the rules without adding to dataset it's also possible as follows

In [ ]:
weak_labels = WeakLabels(rules=rules, dataset="news")

On our machine it took around 24 seconds to apply the rules and to generate weak labels for the 127,600 examples.

Typically, you want to iterate on the rules and check their statistics. 
For this, you can use `weak_labels.summary` method:

In [ ]:
weak_labels.summary()


From the above, we see that our rules cover around **30% of the original training set** with an **average precision of 0.73**. Our hope is that the label and downstream models will improve both the recall and the precision of the final classifier.

## Appendix II: Log datasets to the Hugging Face Hub

Here we will show you how we pushed our Argilla datasets (records) to the [Hugging Face Hub](https://huggingface.co/datasets).
In this way you can effectively version any of your Argilla datasets.

In [ ]:
train_rg = rg.DatasetForTextClassification(label_model.predict())
train_rg.to_datasets().push_to_hub("argilla/news")


In [ ]:
test_rg = rg.load("news", query="status:Validated")
test_rg.to_datasets().push_to_hub("argilla/news_test")
